In [0]:
DEBUG = False

GALAXY = 'booi'
GALAXYFULLNAME = 'bootes'
NVISITS = 1

DATA_DIR = '/datascope/subaru'
ISOCHRONES_PATH = f'{DATA_DIR}/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

OUTPUT_PATH = f'{DATA_DIR}/data/targeting/dSph/{GALAXYFULLNAME}/netflow/{GALAXYFULLNAME}_{NVISITS}'

# Plot the assigned targets

Load the data and plot the assigned targets for each pointing and visit.

In [0]:
import os, sys
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(Pointing(galaxy.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the assignments

In [0]:
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer

In [0]:
assignments = DataFrameSerializer().read(f'{OUTPUT_PATH}/{GALAXY}_assignments.feather')

print(assignments.shape)
for c in assignments.columns:
    print(c, assignments[c].dtype)

In [0]:
assignments['pointing_idx'].unique(), assignments['visit_idx'].unique()

In [0]:
summary = DataFrameSerializer().read(f'{OUTPUT_PATH}/{GALAXY}_summary.feather')
summary.shape

# Load the target lists

In [0]:
from pfs.ga.targeting.config import NetflowConfig
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
# Load the configuration
fn = glob(os.path.join(OUTPUT_PATH, 'ga-netflow*.config'))[0]
config = NetflowConfig.from_file(fn, format='.yaml', ignore_collisions=True)

print(config.targets.keys())

In [0]:
s = ObservationSerializer()

target_lists = {}
for k in config.targets:
    fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_targets_{k}.feather')
    target_lists[k] = s.read(fn)
    print(k, target_lists[k].data.shape, fn)

# Plot all targets

In [0]:
def plot_assignments(axs,
                     pointing_idx=None, visit_idx=None,
                     plot_background=False, plot_assigned=True, plot_unassigned=True,
                     title=None):

    mask = ((pointing_idx is None) | (assignments['pointing_idx'] == pointing_idx)) \
         & ((visit_idx is None) | (assignments['visit_idx'] == visit_idx))

    unique_targets = assignments['targetid'][mask].unique()
    # print(unique_targets.shape)

    cmap = plt.get_cmap('tab10')

    # Plot the full list of science targets in the background in grey
    if plot_background:
        for k, target_list in target_lists.items():

            if target_list.has_diagram_values(cmd, observed=True):
                cmd.plot_catalog(axs[0], target_list, observed=True, color='grey', alpha=1.0)

            if target_list.has_diagram_values(ccd, observed=True):
                ccd.plot_catalog(axs[1], target_list, observed=True, color='grey', alpha=1.0)

            if config.targets[k].prefix == 'sci':
                fov.plot_catalog(axs[2], target_list, observed=True, color='grey', alpha=1.0)

    if plot_unassigned:
        for k, target_list in target_lists.items():
            if config.targets[k].prefix == 'sci':
                mask = ~target_list.data['targetid'].isin(unique_targets)
                # print(k, config.targets[k].prefix, mask.sum())

                for d, ax in zip([cmd, ccd, fov], axs):
                    if target_list.has_diagram_values(d, observed=True):
                        d.plot_catalog(ax, target_list, mask=mask, observed=True, color='grey', size=0.5, alpha=1.0)

    # Plot the assigned targets in color
    if plot_assigned:
        for i, (k, target_list) in enumerate(target_lists.items()):
            mask = target_list.data['targetid'].isin(unique_targets)
            # print(k, config.targets[k].prefix, mask.sum())

            if target_list.has_diagram_values(cmd, observed=True):
                cmd.plot_catalog(axs[0], target_list, mask=mask, observed=True, size=0.5, color=cmap(i))

            if target_list.has_diagram_values(ccd, observed=True):
                ccd.plot_catalog(axs[1], target_list, mask=mask, observed=True, size=0.5, color=cmap(i))

            fov.plot_catalog(axs[2], target_list, mask=mask, observed=True, size=0.5, color=cmap(i), label=k)

    axs[2].set_aspect('equal', adjustable='datalim')
    axs[2].legend(loc='upper right', fontsize=6)

In [0]:
f = plt.figure(figsize=(9, 7), dpi=240)
gs = f.add_gridspec(2, 3, width_ratios=[2, 2, 3], wspace=0.4, hspace=0.2)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]
plot_assignments(axs, plot_background=False, plot_assigned=False, plot_unassigned=True)

axs = [f.add_subplot(gs[1, 0]), f.add_subplot(gs[1, 1]), f.add_subplot(gs[1, 2], projection=wcs.wcs)]
plot_assignments(axs, plot_background=False, plot_assigned=True, plot_unassigned=False)

In [0]:
for i in assignments['pointing_idx'].unique():
    f = plt.figure(figsize=(9, 4), dpi=240)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

    axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

    plot_assignments(axs, pointing_idx=i,
                     plot_background=True, plot_assigned=True, plot_unassigned=False)

    f.suptitle(f'Pointing {i}')

# Plot science targets per priority

In [0]:
#####

# Calculate statistics

In [0]:
from pfs.datamodel import TargetType

In [0]:
# Total number of unique science targets observed
assignments[assignments['prefix'] == 'sci']['targetid'].nunique()

In [0]:
# Targets that are partially observed
summary[(summary['num_visits'] < summary['req_visits']) & (summary['num_visits'] > 0)]

In [0]:
# Science targets that are observed longer than the required number of visits
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] > summary['req_visits'])]

In [0]:
# Number of unique targets per priority class
summary[summary['num_visits'] > 0].groupby('class')['targetid'].nunique()

In [0]:
# Number of missed science targets per priority class
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] == 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of partially observed science targets per priority class
summary[(summary['prefix'] == 'sci') & 
        (summary['num_visits'] < summary['req_visits']) & 
        (summary['num_visits'] > 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of unassigned fibers in each visit
assignments[assignments['target_type'] == TargetType.UNASSIGNED].groupby('visit_idx')['fiberid'].nunique()

In [0]:
# Number of unique calibration targets for each visit
assignments[assignments['prefix'] == 'cal'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of unique sky positions for each visit
assignments[assignments['prefix'] == 'sky'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of observed unique science targets for each visit
assignments[assignments['prefix'] == 'sci'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# of unique science targets assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sci')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sci']['targetid'].nunique())

In [0]:
# total # of sky fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sky')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sky']['targetid'].nunique())

In [0]:
# total # of calibration fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'cal')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'cal']['targetid'].nunique())

# Plot the distribution of assigned targets

In [0]:
# Plot priority distribution for each target list
for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount(target_lists[k].data['priority'])
        ax.bar(np.arange(hist.size), hist)

        mask = target_lists[k].data['targetid'].isin(assignments[assignments['prefix'] == config.targets[k].prefix]['targetid'])
        hist = np.bincount(target_lists[k].data['priority'][mask])
        ax.bar(np.arange(hist.size), hist, color='r')

        ax.set_title(f'Priority class distribution for target list `{k}`')
        ax.set_xlabel(f'Priority class')
        ax.set_ylabel(f'Target count')

In [0]:
# Plot the distribution of required visits for each target list
exp_time = 1800 # seconds

for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount((np.ceil(target_lists[k].data['exp_time'] / exp_time)).astype(int))
        ax.bar(np.arange(hist.size), hist)

        mask = target_lists[k].data['targetid'].isin(assignments[assignments['prefix'] == config.targets[k].prefix]['targetid'])
        hist = np.bincount((np.ceil(target_lists[k].data['exp_time'][mask] / exp_time)).astype(int))
        ax.bar(np.arange(hist.size), hist, color='r')

        ax.set_title(f'Distribution of required visit (t_exp = {exp_time} s)')
        ax.set_xlabel('Number of required visits')
        ax.set_ylabel('Target count')
